In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    
    accuracy = accuracy_score(y_test , pred)
    
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    
    f1 = f1_score(y_test,pred)
    
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
    
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_df = pd.read_csv('./titanic/titanic_train.csv')
predict_df = pd.read_csv('./titanic/test.csv')
gender_submission_df = pd.read_csv('./titanic/gender_submission.csv')

# 알파벳 뒤에 붙는 숫자 값은 무시하고 Alphabet만 가져오는 전략을 선택
train_df['Cabin'] = train_df['Cabin'].str[:1]

def get_category_age(age):
    cat = ''
    if age <= -1: cat = 0
    elif age <= 5: cat = 1
    elif age <= 12: cat = 2
    elif age <= 18: cat = 3
    elif age <= 25: cat = 4
    elif age <= 35: cat = 5
    elif age <= 60: cat = 6
    else: cat = 7
               
    return cat

group_names = [0, 1, 2, 3, 4, 5, 6, 7]
train_df['Age_range'] = train_df['Age'].apply(lambda x : get_category_age(x))

def get_category_fare(fare):
    cat = ''
    if fare <= 0: cat = 0
    elif fare <= 10: cat = 1
    elif fare <= 20: cat = 2
    elif fare <= 30: cat = 3
    elif fare <= 40: cat = 4
    elif fare <= 50: cat = 5
    elif fare <= 60: cat = 6
    else: cat = 7
               
    return cat

group_names = [0, 1, 2, 3, 4, 5, 6, 7]
train_df['Fare_range'] = train_df['Fare'].apply(lambda x : get_category_fare(x))

# 불필요한 특성, null 값 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket', 'Age', 'Fare'], axis=1, inplace=True)

    y = df['Survived']
    df = df.drop('Survived', axis=1, inplace=False)
    return df, y

def fillna(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    return df
# predict_df에서 사용
def drop_features_update(df):
    df.drop(['PassengerId','Name','Ticket', 'Age', 'Fare'], axis=1, inplace=True)
    return df

train_df = fillna(train_df)
X, y = drop_features(train_df)


# 라벨 인코딩 구현
# Sex
def get_category_sex(sex):
    cat = ''
    if sex == "male": cat = 0
    else: cat = 1
    return cat

group_names = [0, 1]
X['Sex'] = X['Sex'].apply(lambda x : get_category_sex(x))

# Embarked
def get_category_embarked(embarked):
    cat = ''
    if embarked == "S": cat = 0
    elif embarked == "C": cat= 1
    elif embarked == "Q": cat= 2
    else: cat = 3
    return cat

group_names = [0, 1, 2, 3]
X['Embarked'] = X['Embarked'].apply(lambda x : get_category_embarked(x))

# Cabin
def get_category_cabin(cabin):
    cat = ''
    if cabin == "A": cat = 0
    elif cabin == "B": cat= 1
    elif cabin == "C": cat= 2
    elif cabin == "D": cat= 3
    elif cabin == "E": cat= 4
    elif cabin == "F": cat= 5
    elif cabin == "G": cat= 6
    elif cabin == "N": cat= 7
    else: cat = 8
    return cat

group_names = [0, 1, 2, 3, 4, 5, 6, 7, 8]
X['Cabin'] = X['Cabin'].apply(lambda x : get_category_cabin(x))

X.head()

,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Age_range,Fare_range
0,3,0,1,0,7,0,4,1
1,1,1,1,0,2,1,6,7
2,3,1,0,0,7,0,5,1
3,1,1,1,0,2,0,5,6
4,3,0,0,0,7,0,5,1


In [9]:
print("train_df 데이터의 행 개수:", len(X))
print('train_df: 데이터 세트 Null 값 갯수 ',X.isnull().sum().sum())
print(X.isnull().sum())
print(X.columns)

train_df 데이터의 행 개수: 891
train_df: 데이터 세트 Null 값 갯수  0
Pclass        0
Sex           0
SibSp         0
Parch         0
Cabin         0
Embarked      0
Age_range     0
Fare_range    0
dtype: int64
Index(['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Age_range',
       'Fare_range'],
      dtype='object')


# 모든 feature 라벨인코딩 -> 스케일링 필요?

> -  순서가 없는 범주형 데이터에 적용하면 문제
> -  거리 기반 알고리즘이나 신경망을 사용할 경우 스케일링이 필요
> -  트리 기반 모델을 사용할 경우에는 필요하지 불필요

1. 거리 기반 알고리즘 (예: K-최근접 이웃(K-NN), K-평균 클러스터링)
    - 필요성: 매우 높음. 피처 간의 거리 계산에 라벨 인코딩된 값이 큰 영향을 줄 수 있으므로, 스케일링이 필요합니다.
2. 선형 모델 (예: 로지스틱 회귀, 선형 회귀)
    - 필요성: 보통 높음. 라벨 인코딩된 값이 모델의 가중치 추정에 영향을 미칠 수 있으므로, 스케일링이 유리할 수 있습니다.
3. 트리 기반 모델 (예: 결정 트리, 랜덤 포레스트, 그라디언트 부스팅)
    - 필요성: 낮음. 트리 기반 모델은 피처의 스케일에 크게 영향을 받지 않으므로, 스케일링이 필요하지 않습니다.
4. 신경망 (예: MLP, RNN, CNN)
    - 필요성: 높음. 신경망의 학습 과정에서 가중치 업데이트의 안정성을 위해 피처 스케일링이 중요합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

# 표준화 => 2차원 데이터
sc = StandardScaler()
X = sc.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
predict_df['Age_range'] = predict_df['Age'].apply(lambda x : get_category_age(x))

predict_df['Fare_range'] = predict_df['Fare'].apply(lambda x : get_category_fare(x))

predict_df = fillna(predict_df)
predict_df = drop_features_update(predict_df)

predict_df['Sex'] = predict_df['Sex'].apply(lambda x : get_category_sex(x))
predict_df['Embarked'] = predict_df['Embarked'].apply(lambda x : get_category_embarked(x))

predict_df['Cabin'] = predict_df['Cabin'].str[:1]
predict_df['Cabin'] = predict_df['Cabin'].apply(lambda x : get_category_cabin(x))

predict_df.head(3)

,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Age_range,Fare_range
0,3,0,0,0,7,2,5,1
1,3,1,1,0,7,0,6,1
2,2,0,0,0,7,2,7,1


In [ ]:
# 스케일링 불필요

predict_df = sc.transform(predict_df)
predict_df[1]

# RandomForest